Installing  Libraries 

In [23]:
%pip install pandas pynance  ta-lib matplotlib

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\amr\project\kifiya\stock-sentiment-correlation-analysis\.venv\lib\site-packages\talib-0.4.33-py3.12-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


Importing Libraries

In [24]:
import pandas as pd
import talib as ta 
import pynance as pn
import matplotlib.pyplot as plt


Loading datasets

In [8]:
data_files = [
    "AAPL_historical_data.csv", "AMZN_historical_data.csv",
    "GOOG_historical_data.csv", "META_historical_data.csv",
    "MSFT_historical_data.csv", "NVDA_historical_data.csv",
    "TSLA_historical_data.csv"
]

data_dict = {}
for file in data_files:
    stock_name = file.split('_')[0]
    data_dict[stock_name] = pd.read_csv(f'./yfinance_data/{file}')
    data_dict[stock_name].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10998 entries, 0 to 10997
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          10998 non-null  object 
 1   Open          10998 non-null  float64
 2   High          10998 non-null  float64
 3   Low           10998 non-null  float64
 4   Close         10998 non-null  float64
 5   Adj Close     10998 non-null  float64
 6   Volume        10998 non-null  int64  
 7   Dividends     10998 non-null  float64
 8   Stock Splits  10998 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 773.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6846 entries, 0 to 6845
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          6846 non-null   object 
 1   Open          6846 non-null   float64
 2   High          6846 non-null   float64
 3   Low           6846 non-null  

Apply Analysis Indicators with TA-Lib

In [12]:
for stock_name, df in data_dict.items():
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

In [14]:
for stock, df in data_dict.items():
    #50-day simple moving average
    df['SMA_50'] = ta.SMA(df['Close'], timeperiod=50)
    #14-day RSI
    df['RSI_14'] = ta.RSI(df['Close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(df['Close'])
    print(f'{stock} with indicators')
    print(df.head())

AAPL with indicators
                Open      High       Low     Close  Adj Close     Volume  \
Date                                                                       
1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   
1980-12-16  0.113281  0.113281  0.112723  0.112723   0.086898  105728000   
1980-12-17  0.115513  0.116071  0.115513  0.115513   0.089049   86441600   
1980-12-18  0.118862  0.119420  0.118862  0.118862   0.091630   73449600   

            Dividends  Stock Splits  SMA_50  RSI_14  MACD  MACD_signal  \
Date                                                                     
1980-12-12        0.0           0.0     NaN     NaN   NaN          NaN   
1980-12-15        0.0           0.0     NaN     NaN   NaN          NaN   
1980-12-16        0.0           0.0     NaN     NaN   NaN          NaN   
1980-12-17        0.0           0.0     NaN     NaN   NaN          NaN   
19

In [20]:
#After calculating indicators dropping a values NaN indicators
for stock, df in data_dict.items():
    df.dropna(subset=['SMA_50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_hist'], inplace=True)
    print(df[50:].head()) 

                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
1981-05-06  0.122768  0.122768  0.122210  0.122210   0.094211  18950400   
1981-05-07  0.123884  0.124442  0.123884  0.123884   0.095502   9363200   
1981-05-08  0.125000  0.125558  0.125000  0.125000   0.096362   7907200   
1981-05-11  0.122768  0.122768  0.122210  0.122210   0.094211  11939200   
1981-05-12  0.122210  0.123884  0.122210  0.122210   0.094211   4256000   

            Dividends  Stock Splits    SMA_50     RSI_14      MACD  \
Date                                                                 
1981-05-06        0.0           0.0  0.116752  50.117378  0.002346   
1981-05-07        0.0           0.0  0.116975  52.583107  0.002113   
1981-05-08        0.0           0.0  0.117187  54.208200  0.001996   
1981-05-11        0.0           0.0  0.117266  49.628854  0.001658   
1981-05-12        0.0           0.0  0.117333  49.6288

Use PyNance for Financial Metrics

In [28]:
# Calculate daily returns manually
for stock_name, df in data_dict.items():
    print(f"Metrics for {stock_name}:")

    # Calculate daily returns for various columns
    df['Open_Return'] = df['Open'].pct_change()
    df['High_Return'] = df['High'].pct_change()
    df['Low_Return'] = df['Low'].pct_change()
    df['Close_Return'] = df['Close'].pct_change()
    df['Adj_Close_Return'] = df['Adj Close'].pct_change()

    # Summary statistics on returns
    print(f"Summary statistics for {stock_name} Returns:")
    print(df[['Open_Return', 'High_Return', 'Low_Return', 'Close_Return', 'Adj_Close_Return']].describe())


Metrics for AAPL:
Summary statistics for AAPL Returns:
        Open_Return   High_Return    Low_Return  Close_Return  \
count  10948.000000  10948.000000  10948.000000  10948.000000   
mean       0.001092      0.001010      0.001056      0.001092   
std        0.028070      0.024933      0.026586      0.027840   
min       -0.428390     -0.461092     -0.472727     -0.518692   
25%       -0.012915     -0.010695     -0.011113     -0.012653   
50%        0.000000      0.000000      0.001186      0.000000   
75%        0.014056      0.011687      0.013119      0.014440   
max        0.266458      0.387504      0.283075      0.332280   

       Adj_Close_Return  
count      10948.000000  
mean           0.001115  
std            0.027837  
min           -0.518692  
25%           -0.012623  
50%            0.000000  
75%            0.014440  
max            0.332281  
Metrics for AMZN:
Summary statistics for AMZN Returns:
       Open_Return  High_Return   Low_Return  Close_Return  Adj_Close_

In [ ]:
# Calculate beta and Sharpe ratio
for stock_name, df in data_dict.items():
    # Benchmark returns example (you would have your benchmark data for this)
    benchmark_returns = df['Close'].pct_change()  # Dummy data for example

    # Beta calculation (requires benchmark)
    beta = pn.analysis.beta(df['Close'].pct_change(), benchmark_returns)
    print(f"Beta for {stock_name}: {beta}")

    # Sharpe ratio
    sharpe_ratio = pn.analysis.sharpe(df['Close'].pct_change())
    print(f"Sharpe Ratio for {stock_name}: {sharpe_ratio}")


Visualize the Data

In [ ]:
stock_names = list(data_dict.keys())
betas = [pn.analysis.beta(df['Close'].pct_change(), benchmark_returns) for df in data_dict.values()]
sharpe_ratios = [pn.analysis.sharpe(df['Close'].pct_change()) for df in data_dict.values()]

plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.bar(stock_names, betas, color='blue')
plt.title('Beta Values')
plt.xlabel('Stock')
plt.ylabel('Beta')

plt.subplot(1, 2, 2)
plt.bar(stock_names, sharpe_ratios, color='green')
plt.title('Sharpe Ratios')
plt.xlabel('Stock')
plt.ylabel('Sharpe Ratio')

plt.tight_layout()
plt.show()